This notebook is associated with the paper "The relative class number one problem for function fields, II" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and Magma (tested using version 2.25-5); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we make computations as part of the proof that for $q=2$, there are no non-Galois covers with relative class number 1. This depends on the exhaustion over Weil polynomials performed in "The relative class number one problem for function fields, I".

In [47]:
load("auxiliary.sage")

In [48]:
P.<T> = QQ[]

Read data about the exhaustion from an Excel spreadsheet.

In [49]:
import pandas

In [50]:
df = pandas.read_excel('polys.xlsx')
print(list(df))

['Unnamed: 0', 'd', 'g', "g'", 'Label of J(C)', '#J(C)', 'Counts of C', "Counts of C'", 'Cyclic']


In [51]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    counts1 = eval(r["Counts of C"])
    counts2 = eval(r["Counts of C'"])
    ordj = r["#J(C)"]
    candidates[d,g,g1].append((counts1, counts2, ordj))

Identify all Weil polynomials with specified initial Frobenius traces.

In [52]:
def weil_polys_from_traces(P, d, q, l):
    Q.<t> = PowerSeriesRing(QQ)
    u = sum(-l[i-1]*t^i/i for i in range(1,len(l)+1))
    v = exp(u)
    l2 = v.polynomial().list()[:len(l)+1]
    l2 = [ZZ(i) for i in l2]
    l3 = P.weil_polynomials(d=d, q=q, lead=l2)
    if len(l2) < d//2+1:
        return l3
    return [w for w in l3 if w.is_weil_polynomial()]

In [53]:
def weil_polys_from_point_counts(P, d, q, l):
    return weil_polys_from_traces(P, d, q, [q^i+1-l[i-1] for i in range(1, len(l)+1)])

Confirm that for $d>2$, the Galois group contains a $d$-cycle.

In [54]:
for (d, g, g1) in candidates:
    if d > 2:
        for (a,b,_) in candidates[d,g,g1]:
            u = weil_poly_from_point_count(a, g)
            v = weil_poly_from_point_count(b, g1)
            ct1 = place_count_from_weil_poly(u, 5)
            ct2 = place_count_from_weil_poly(v, 5)
            if not (ct1[0] > sum(ct2[i] for i in range((d-1)//2)) + (0 if d%2 else 1/2*ct2[d//2-1])):
                print(d,g,g1,ct1,ct2)

5 2 6 [5, 2, 0, 2, 4] [0, 5, 0, 0, 5]


Case $d=3$.

In [55]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,4,10]]

[(6, 6, 0, 0), (6, 6, 0, 0), (7, 7, 0, 0), (7, 9, 0, 2), (8, 8, 0, 0)]

In [56]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,3,7] if b[0] > 1]

[(6, 10, 2, 2)]

In [57]:
[(a[0], a[1], a[2], b[0], b[1], b[2]) for (a,b,_) in candidates[3,3,7] if b[0] == 1 and b[1] >= 3]

[(5, 9, 14, 1, 3, 16)]

In [58]:
assert not weil_polys_from_traces(P, 4, 2, [-3, -3, 0])

In [59]:
[(a[0], a[1], b[0], b[1]) for (a,b,_) in candidates[3,3,7] if b[0] == 0]

[(2, 8, 0, 0),
 (3, 7, 0, 0),
 (3, 7, 0, 0),
 (4, 6, 0, 0),
 (4, 6, 0, 0),
 (4, 8, 0, 0),
 (4, 8, 0, 0),
 (4, 8, 0, 0),
 (4, 10, 0, 2),
 (4, 12, 0, 6)]

In [60]:
assert not weil_polys_from_traces(P, 4, 2, [-2, -8])

In [61]:
assert not weil_polys_from_traces(P, 4, 2, [-3, -7])

In [62]:
assert not weil_polys_from_traces(P, 4, 2, [-4, -6])

In [63]:
assert not weil_polys_from_traces(P, 4, 2, [-4, -8])

In [64]:
[(a[0], b[0]) for (a,b,_) in candidates[3,2,4]]

[(1, 0), (3, 0), (4, 0)]

In [65]:
[(a[0], b[0], b[1]) for (a,b,_) in candidates[3,2,6]]

[(3, 0, 2), (4, 0, 2), (4, 0, 2), (5, 0, 2)]

Case $d=4$.

In [66]:
for (d, g, g1) in [(4,3,9), (4,2,5)]:
    for (a,b,_) in candidates[d,g,g1]:
        for i in range(len(df)):
            if df.loc[i, "d"] == 2 and df.loc[i, "g"] == g and \
            df.loc[i, "g'"] == 2*g-1 and eval(df.loc[i, "Counts of C"]) == a and \
            df.loc[i, "Cyclic"] == "Yes":
                for j in range(len(df)):
                    if df.loc[j, "d"] == 2 and df.loc[j, "g"] == 2*g-1 and \
                    df.loc[j, "g'"] == g1 and df.loc[j, "Cyclic"] == "Yes" and\
                    df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                    eval(df.loc[j, "Counts of C'"]) == b:
                        print(d,g,g1,a,b)

4 2 5 [2, 8, 8, 24, 52, 56, 100, 256, 476, 968, 2180, 4272, 8140] [0, 0, 12, 16, 60, 24, 84, 160, 444, 840, 2244, 4144, 8268]


In [67]:
[(a[0], a[1], b[0], b[1]) for (a,b,c) in candidates[4,3,9] if c%3 == 0]

[(5, 9, 0, 0)]

In [68]:
[(a[0], b[0], b[1], b[2]) for (a,b,_) in candidates[4,3,9]]

[(5, 0, 0, 0), (6, 0, 0, 6), (6, 0, 2, 0), (6, 0, 0, 0), (6, 0, 2, 0)]

In [69]:
[(a[0], b[0]) for (a,b,c) in candidates[4,2,5] if c%3 == 0]

[(4, 1), (5, 1)]

In [70]:
[(a[0], b[0], b[1], b[2], c) for (a,b,c) in candidates[4,2,5] if c%2 == 0]

[(2, 0, 0, 12, 4),
 (4, 0, 4, 12, 8),
 (4, 1, 1, 10, 8),
 (4, 0, 8, 6, 10),
 (4, 1, 5, 4, 10)]

Case $d=5$.

In [71]:
[(a[:6], b[:6], c) for (a,b,c) in candidates[5,2,6]]

[([3, 5, 9, 33, 33, 65], [0, 0, 0, 20, 15, 90], 5),
 ([3, 7, 9, 31, 33, 43], [0, 0, 9, 8, 30, 33], 6),
 ([4, 8, 10, 24, 14, 56], [0, 0, 15, 20, 20, 45], 10),
 ([4, 8, 10, 24, 14, 56], [0, 6, 0, 18, 0, 60], 10),
 ([4, 10, 7, 18, 24, 55], [0, 2, 15, 18, 40, 23], 11),
 ([5, 9, 5, 17, 25, 81], [0, 6, 3, 34, 30, 63], 15),
 ([5, 9, 5, 17, 25, 81], [0, 10, 0, 10, 25, 70], 15),
 ([6, 6, 9, 10, 36, 87], [1, 3, 7, 27, 41, 69], 19)]

In [72]:
set(trace_from_weil_poly(u, 5)[4] for u in weil_polys_from_traces(P, 12, 2, [-3, -5, -9]))

{-13, -8, -3, 2, 7, 12, 17, 22, 27, 32, 37, 47, 57}

In [73]:
assert not weil_polys_from_traces(P, 12, 2, [-3,-7,0,-27,-18])

In [74]:
assert not weil_polys_from_traces(P, 12, 2, [-3,-7,0,-15,-18])

In [75]:
assert not weil_polys_from_traces(P, 12, 2, [-4,-8,14,-52])

In [76]:
assert not weil_polys_from_traces(P, 12, 2, [-5,-19,-5])

Case $d=6$.

In [77]:
for (a,b,_) in candidates[6,2,7]:
    for i in range(len(df)):
        if df.loc[i, "d"] == 2 and df.loc[i, "g"] == 2 and \
        df.loc[i, "g'"] == 3 and eval(df.loc[i, "Counts of C"]) == a and \
        df.loc[i, "Cyclic"] == "Yes":
            for j in range(len(df)):
                if df.loc[j, "d"] == 3 and df.loc[j, "g"] == 3 and \
                df.loc[j, "g'"] == 7 and df.loc[j, "Cyclic"] == "Yes" and\
                df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                eval(df.loc[j, "Counts of C'"]) == b:
                    print(a,b)

In [78]:
l = []
for (a,b,c) in candidates[6,2,7]:
    u1 = weil_poly_from_point_count(a, 2)
    u2 = weil_poly_from_point_count(b, 7)
    v1 = u1.trace_polynomial()[0]
    v2 = (u2//u1).trace_polynomial()[0]
    if modified_reduced_resultant(v1, v2)%3 != 0:
        l.append((a, b, c))
[(a[:6], b[:6], c) for (a,b,c) in l]

[([5, 5, 17, 9, 25, 65], [0, 0, 3, 12, 35, 87], 13),
 ([5, 9, 5, 17, 25, 81], [0, 2, 9, 2, 30, 71], 15),
 ([5, 9, 5, 17, 25, 81], [0, 4, 0, 12, 15, 106], 15),
 ([6, 6, 9, 10, 36, 87], [0, 0, 15, 20, 30, 87], 19)]

In [79]:
[(a[:6], b[:6], c) for (a,b,c) in candidates[6,2,7] if (a,b,c) not in l]

[([4, 8, 10, 24, 14, 56], [0, 0, 6, 8, 30, 24], 10),
 ([5, 5, 17, 9, 25, 65], [0, 0, 12, 4, 15, 90], 13),
 ([5, 5, 17, 9, 25, 65], [0, 2, 3, 10, 30, 47], 13),
 ([5, 7, 11, 15, 15, 91], [0, 2, 6, 10, 5, 116], 14),
 ([5, 9, 5, 17, 25, 81], [1, 1, 1, 1, 41, 49], 15),
 ([6, 6, 9, 10, 36, 87], [0, 2, 3, 14, 35, 131], 19),
 ([6, 6, 9, 10, 36, 87], [0, 4, 3, 12, 30, 91], 19)]

In [80]:
assert not weil_polys_from_traces(P, 10, 2, [0,-4,-15,4,0])

In [81]:
assert not weil_polys_from_traces(P, 10, 2, [0,-4,-15,-20,0])

In [82]:
assert not weil_polys_from_traces(P, 10, 2, [0,-4,-15,-8,0])

In [83]:
l = []
for i in [-3, -12, -21]:
    for j in [15, 5, 0, -10]:
        l2 = weil_polys_from_traces(P, 10, 2, [0, -10, i, -10, j])
        if l2:
            print(i,j)
            l += l2

-3 15


In [84]:
l

[T^10 + 5*T^8 + T^7 + 15*T^6 + 2*T^5 + 30*T^4 + 4*T^3 + 40*T^2 + 32]

In [85]:
trace_from_weil_poly(l[0], 6)[5]

23

In [86]:
assert not weil_polys_from_traces(P, 10, 2, [0,-10,-15,-10])

Case $d=7$.

In [87]:
(d, g, g1) = (7, 2, 8)
candidates[d, g, g1]

[([5, 7, 11, 15, 15, 91, 131, 255, 479, 987, 2227, 3999, 8143],
  [0, 0, 0, 0, 0, 84, 133, 336, 378, 980, 2310, 4032, 8008],
  14)]

In [88]:
u = weil_poly_from_point_count(candidates[d, g, g1][0][0], g); u

T^4 + 2*T^3 + 3*T^2 + 4*T + 4

In [89]:
v = weil_poly_from_point_count(candidates[d, g, g1][0][1], g1); v

T^16 - 3*T^15 + 2*T^14 + 14*T^10 - 23*T^9 + 13*T^8 - 46*T^7 + 56*T^6 + 128*T^2 - 384*T + 256

In [90]:
K1 = magma.NumberField(u)
a  = K1.gen(1)
O1 = K1.MaximalOrder()
O1a = O1.sub([a, 2/a])
assert O1a == O1
assert O1.ClassNumber() == 1

In [91]:
K2 = magma.NumberField(v//u)
b = K2.gen(1)
O2 = K2.MaximalOrder()
O2a = O2.sub([b, 2/b])
assert O2a == O2
assert O2.ClassNumber() == 1

In [92]:
K2s = K2.sage()
l = K2s.primes_above(7)
assert(len(l) == 1)
p = l[0]
K2s(7).valuation(p)

6